In [1]:
#ライブラリの自動リロード
%load_ext autoreload
%autoreload 2

from scoring import eval_model
from transformers import pipeline
import os


/home/hatakeyama/miniconda3/envs/ft/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
mode="qlora"
model_name = "meta-llama/Llama-2-7b-chat-hf"

In [3]:
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import os

os.environ["HF_HOME"]="/media/hatakeyama/hf"

#load base model
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, 
                                            quantization_config=bnb_config, 
                                            device_map="auto",
                                            )


Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.29s/it]


In [4]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm

In [4]:
pipe= pipeline("text-generation", model=model, tokenizer=tokenizer,max_new_tokens=100)

In [5]:
import json
dataset_path="../database/output/qa_dataset.json"
with open(dataset_path, "r") as f:
    raw_dataset = json.load(f)

spl_pos=200

# train

In [6]:
import transformers
from datasets import load_dataset,Dataset
from scoring import generate_prompt
import random

#context
context_path="../database/output/context0926.json"
with open(context_path, 'r') as f:
    context_list = json.load(f)

data_list=[{"text":i} for i in context_list]

#q and a
q_a_data_list=[{"text": generate_prompt(d)} for d in raw_dataset[spl_pos:]]
# listデータをDatasetに変換

data_list.extend(q_a_data_list)
random.shuffle(data_list)


dataset = Dataset.from_dict({"text": [item["text"] for item in data_list]})

In [7]:

train_dataset=dataset.map(lambda samples: tokenizer(samples['text']), batched=True)
tokenizer.pad_token = tokenizer.eos_token


Map: 100%|██████████| 86107/86107 [00:07<00:00, 11131.20 examples/s]


In [8]:
from peft import LoraConfig, get_peft_model
per_device_train_batch_size=1
epochs=1

train_args=transformers.TrainingArguments(
        per_device_train_batch_size=per_device_train_batch_size,
        gradient_accumulation_steps=1,
        warmup_steps=100,
        num_train_epochs=epochs,  # エポック数
        #max_steps=tot_steps,
        learning_rate=2e-5,
        fp16=True,
        logging_steps=100,
        output_dir='outputs',
        #optim=""
    )

peft_config = LoraConfig(
        task_type="CAUSAL_LM", inference_mode=False, r=8, lora_alpha=32,
        lora_dropout=0.1
    )
model = get_peft_model(model, peft_config)

trainer = transformers.Trainer(
    model=model,
    train_dataset=train_dataset,
    args=train_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)



#model.config.use_cache = True  # silence the warnings. Please re-enable for inference!
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


KeyboardInterrupt: 

In [ ]:
peft_name = f"model/all_{mode}_qa"
trainer.model.save_pretrained(peft_name)
tokenizer.save_pretrained(peft_name)

('model/10000_qlora/tokenizer_config.json',
 'model/10000_qlora/special_tokens_map.json',
 'model/10000_qlora/tokenizer.model',
 'model/10000_qlora/added_tokens.json',
 'model/10000_qlora/tokenizer.json')

In [1]:

eval_model(raw_dataset[:200],pipe,f"res/0929test/10000_{mode}_qa_200.csv")

NameError: name 'eval_model' is not defined